<a href="https://colab.research.google.com/github/sohdacafe/Machine_Learning/blob/main/deci_tree_breast_cancer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [8]:
from sklearn.datasets import  load_breast_cancer

In [ ]:
dataset = load_breast_cancer()
dataset

In [14]:
t = dataset.target
x = dataset.data

In [15]:
x.shape, t.shape

((569, 30), (569,))

In [16]:
from sklearn.model_selection import train_test_split
x_train_val, x_test, t_train_val, t_test = train_test_split(x, t, test_size=0.2, random_state=1)

In [17]:
x_train_val.shape, x_test.shape

((455, 30), (114, 30))

In [18]:
x_train, x_val, t_train, t_val = train_test_split(x_train_val, t_train_val, test_size=0.3, random_state=1)

In [19]:
x_train.shape, x_val.shape, x_test.shape

((318, 30), (137, 30), (114, 30))

In [23]:
from sklearn.tree import DecisionTreeClassifier
dtree = DecisionTreeClassifier(max_depth=10, min_samples_split=30, random_state=0)
dtree.fit(x_train, t_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=10, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=30,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=0, splitter='best')

In [25]:
print(f'train score: {dtree.score(x_train, t_train)}')
print(f'val score: {dtree.score(x_val, t_val)}')
print(f'test score: {dtree.score(x_test, t_test)}')

train score: 0.9308176100628931
val score: 0.9562043795620438
test score: 0.9298245614035088


In [ ]:
# グリッドサーチ、ハイパーパラメータの調整方法

In [26]:
from sklearn.model_selection import GridSearchCV

In [40]:
estimator = DecisionTreeClassifier(random_state = 0)
param_grid = [{'max_depth' : [3,20,50], 'min_samples_split':[3,20,30]}]
cv =5

In [41]:
tuned_model = GridSearchCV(estimator = estimator, param_grid=param_grid, cv=cv, return_train_score=False)
tuned_model.fit(x_train_val, t_train_val)

GridSearchCV(cv=5, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=0, splitter='best'),
             iid='deprecated', n_jobs=None,
             param_grid=[{'max_depth': [3, 20, 50],
                          'min_samples_split': [3, 20, 30]}],
             

In [52]:
# 学習結(果はcv_resultsに格納されている
df = pd.DataFrame(tuned_model.cv_results_)
df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_min_samples_split,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.003854,0.000277,0.000326,0.000041,3,3,"{'max_depth': 3, 'min_samples_split': 3}",0.923077,0.901099,0.934066,0.945055,0.901099,0.920879,0.017582,5
1,0.004152,0.000517,0.000316,0.000012,3,20,"{'max_depth': 3, 'min_samples_split': 20}",0.912088,0.901099,0.934066,0.934066,0.901099,0.916484,0.014906,8
2,0.003755,0.000101,0.000329,0.000039,3,30,"{'max_depth': 3, 'min_samples_split': 30}",0.912088,0.901099,0.934066,0.934066,0.901099,0.916484,0.014906,8
3,0.004461,0.000206,0.000332,0.000034,20,3,"{'max_depth': 20, 'min_samples_split': 3}",0.956044,0.912088,0.923077,0.967033,0.956044,0.942857,0.021308,1
4,0.004344,0.000238,0.000292,0.000007,20,20,"{'max_depth': 20, 'min_samples_split': 20}",0.912088,0.901099,0.934066,0.945055,0.934066,0.925275,0.016150,3
5,0.005115,0.001633,0.000334,0.000035,20,30,"{'max_depth': 20, 'min_samples_split': 30}",0.912088,0.901099,0.934066,0.945055,0.901099,0.918681,0.017855,6
6,0.004492,0.000257,0.000314,0.000012,50,3,"{'max_depth': 50, 'min_samples_split': 3}",0.956044,0.912088,0.923077,0.967033,0.956044,0.942857,0.021308,1
7,0.004385,0.000224,0.000300,0.000008,50,20,"{'max_depth': 50, 'min_samples_split': 20}",0.912088,0.901099,0.934066,0.945055,0.934066,0.925275,0.016150,3
8,0.005185,0.000846,0.000424,0.000138,50,30,"{'max_depth': 50, 'min_samples_split': 30}",0.912088,0.901099,0.934066,0.945055,0.901099,0.918681,0.017855,6


In [51]:
df['mean_test_score']

0    0.920879
1    0.916484
2    0.916484
3    0.942857
4    0.925275
5    0.918681
6    0.942857
7    0.925275
8    0.918681
Name: mean_test_score, dtype: float64

In [53]:
tuned_model.best_params_

{'max_depth': 20, 'min_samples_split': 3}

In [55]:
best_model = tuned_model.best_estimator_

In [57]:
print(best_model.score(x_train_val, t_train_val))
print(best_model.score(x_test, t_test))

1.0
0.9385964912280702
